In [ ]:
import time
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import skdim
import random
import numpy as np
import math
import pandas as pd
import keras
import cv2
import skdim
import tensorflow as tf
import seaborn as sns

from tensorflow.keras.utils import to_categorical
from collections import Counter
from PIL import Image
#from google.colab.patches import cv2_imshow
from tensorflow.keras.applications import MobileNetV2
from matplotlib import pyplot as plt
from skopt import gp_minimize
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.neighbors import NearestNeighbors
from itertools import combinations
from tqdm.notebook import tqdm
from scipy import stats
from sklearn.linear_model import Lasso, lars_path, Ridge, ElasticNet, LogisticRegression, SGDClassifier
from collections import Counter
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.stats import wasserstein_distance

from scipy.stats import mannwhitneyu
from scipy.stats import f_oneway
from scipy.stats import ttest_ind
from scipy.stats import wilcoxon

In [ ]:
# Import data and convert to data and label
def import_data(folder_path, train):
  """
  input:
    dataset is whether you want to get the training or test data
    folder_path is the link to the file containing the image data (e.g."XXX")
    target_size is the desired size after resizing the images (e.g. 224*224 pixels)
  output:
    images_array is the array containing the pixel values of the images with shape (8, 224, 224, 3) and value range in [0,255]
    str_label_array is the array containing the string labels of the images
    int_label_array is the array containing the integer labels of the images
  """

  if train==1:
    folder_path = os.path.join(folder_path, "Train 2800")
    print('Importing Training data...')
  elif train == 2:
    print('Importing Explanation data...')
    folder_path = os.path.join(folder_path, "Pool 25")
  elif train == 0:
    print('Importing Test data...')
    folder_path = os.path.join(folder_path, "Test 700")  

  class_folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

  # Initialize an empty list to store pixel values
  images = []
  ground_truth_labels = []
  image_names = []

  for class_folder in class_folders:
        class_folder_path = os.path.join(folder_path, class_folder)
        image_files = [f for f in os.listdir(class_folder_path) if f.endswith('.jpg') or f.endswith('.png')]

        for image_file in image_files:
            image_path = os.path.join(class_folder_path, image_file)
            image_names.append(image_file)
            img = cv2.imread(image_path)
            #resized_img = cv2.resize(img, target_size)
            images.append(img)
            ground_truth_labels.append(class_folder)

  images_array = np.array(images)
  print('Imported', images_array.shape[0], 'images of shape', images_array.shape[1:4])


  str_ground_truth_labels = np.array(ground_truth_labels)

  label_mapping = {"Tel-Aviv": "TelAviv",
          "West Jerusalem": "Jerusalem",
          "WestJerusalem": "Jerusalem",
          "Hamburg": "Hamburg",
          "Berlin": "Berlin"}

  # Map original class labels to new label names
  str_ground_truth_labels = np.array([label_mapping[label] for label in str_ground_truth_labels])
  print('Remapped to the following classes: ', np.unique(str_ground_truth_labels, return_counts=True)[0])
  print('Found', np.unique(str_ground_truth_labels, return_counts=True)[1], 'examples for the different classes respectively')

  # Assuming you have a function strLabel_to_intLabel_mapping that converts string labels to integers
  int_ground_truth_labels = strLabel_to_intLabel_mapping(str_ground_truth_labels)

  cat_ground_truth_labels = to_categorical(int_ground_truth_labels, 4)


  return images_array, int_ground_truth_labels, cat_ground_truth_labels, str_ground_truth_labels, image_names

# Map string labels (e.g. "Jerusalem") to integer labels (e.g. 1)
def strLabel_to_intLabel_mapping(y):
  """
  input:
    y is the array of string labels
  output:
    int_labels_mapped is the array of the corresponding integer labels
  """''
  # Create a dictionary to map string labels to int labels
  label_mapping = {'TelAviv': 2, 'Jerusalem': 3, 'Berlin': 0, 'Hamburg': 1}
  # Map string labels to int labels using the created dictionary
  int_labels_mapped = np.array([label_mapping[val] for val in y])
  return int_labels_mapped

In [ ]:
X_train, y_int_train, y_cat_train, y_str_train, image_names_train = import_data("XXX", train = 1)

In [ ]:
plt.imshow(cv2.cvtColor(X_train[0], cv2.COLOR_BGR2RGB))

In [ ]:
X_test, y_int_test, y_cat_test, y_str_test, image_names_test = import_data("XXX", train = 0)

In [ ]:
plt.imshow(cv2.cvtColor(X_test[0], cv2.COLOR_BGR2RGB))

In [ ]:
X_expl, y_int_expl, y_cat_expl, y_str_expl, image_names_expl = import_data("XXX", train = 2)

In [ ]:
plt.figure(figsize=(12, 12))

for i in range(16):
    ax = plt.subplot(4, 4, i + 1)
    i = random.randint(0,99)
    plt.imshow(cv2.cvtColor(X_expl[i], cv2.COLOR_BGR2RGB))
    plt.title(y_str_expl[i])
    plt.axis("off")
    
plt.suptitle('Random examples of study images', fontsize=13)
plt.tight_layout()

plt.savefig('studyimages.png', dpi=300)

In [ ]:
IMG_SIZE = (448, 448)
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

base_model.trainable = False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(4, activation='softmax')
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

inputs = tf.keras.Input(shape=(448, 448, 3))
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
model.load_weights('fine_train_on_array.keras')

In [ ]:
pred_test = np.argmax(model.predict(X_test), axis = 1)

In [ ]:
accuracy = accuracy_score(y_int_test, pred_test)
print(f"Accuracy: {accuracy:.4f}")

class_report = classification_report(y_int_test, pred_test)
print("Classification Report:")
print(class_report)

conf_matrix = confusion_matrix(y_int_test, pred_test)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
pred_expl = np.argmax(model.predict(X_expl), axis = 1)

In [ ]:
accuracy = accuracy_score(y_int_expl, pred_expl)
print(f"Accuracy: {accuracy:.4f}")

class_report = classification_report(y_int_expl, pred_expl)
print("Classification Report:")
print(class_report)

conf_matrix = confusion_matrix(y_int_expl, pred_expl)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
model.summary()

In [ ]:
IMG_SIZE = (448, 448)
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

base_model.trainable = False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(4, activation='softmax')
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

inputs = tf.keras.Input(shape=(448, 448, 3))
x = preprocess_input(inputs)
x = base_model(x, training=False)

grad_model = tf.keras.Model(inputs, x)

In [ ]:
grad_model.summary()

In [ ]:
grad_model.load_weights('fine_train_on_array.keras')

In [ ]:
weights = model.get_weights()

for i, weight in enumerate(weights):
    print(f"Layer {i} weights shape: {weight.shape}")
    print(weight)

In [ ]:
weights = grad_model.get_weights()

for i, weight in enumerate(weights):
    print(f"Layer {i} weights shape: {weight.shape}")
    print(weight)

In [ ]:
layer_names = [layer.name for layer in model.layers]
classifier_layer_names = layer_names[-3:]
print(classifier_layer_names)

In [ ]:
def gradCAM(img_array, model, grad_conv_model, classifier_layer_names):
    
    classifier_input = tf.keras.Input(shape=grad_conv_model.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)
    classifier_model = keras.Model(classifier_input, x)   
      
    with tf.GradientTape() as tape:
        last_conv_layer_output = grad_conv_model(img_array)
        tape.watch(last_conv_layer_output)
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]
    grads = tape.gradient(top_class_channel, last_conv_layer_output) 
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]
        
    heatmap = np.mean(last_conv_layer_output, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    
    
    return heatmap

In [ ]:
def gradCAMplusplus(img_array, model, grad_conv_model, classifier_layer_names):
    classifier_input = tf.keras.Input(shape=grad_conv_model.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)
    classifier_model = keras.Model(classifier_input, x)
        
    with tf.GradientTape() as tape1:
        with tf.GradientTape() as tape2:
            with tf.GradientTape() as tape3:
                last_conv_layer_output = grad_conv_model(img_array)
                #tape.watch(last_conv_layer_output)
                preds = classifier_model(last_conv_layer_output)
                
                top_pred_index = tf.argmax(preds[0])
                top_class_channel = preds[:, top_pred_index]
                
                conv_first_grad = tape3.gradient(top_class_channel, last_conv_layer_output)
            conv_second_grad = tape2.gradient(conv_first_grad, last_conv_layer_output)
        conv_third_grad = tape1.gradient(conv_second_grad, last_conv_layer_output)
    
    global_sum = np.sum(last_conv_layer_output, axis=(0, 1, 2))
    
    alpha_num = conv_second_grad[0]
    alpha_denom = conv_second_grad[0]*2.0 + conv_third_grad[0]*global_sum
    alpha_denom = np.where(alpha_denom != 0.0, alpha_denom, 1e-10)
    
    alphas = alpha_num/alpha_denom
    alpha_normalization_constant = np.sum(alphas, axis=(0,1))
    alphas /= alpha_normalization_constant
    
    weights = np.maximum(conv_first_grad[0], 0.0)
    
    deep_linearization_weights = np.sum(weights*alphas, axis=(0,1))
    grad_cam_map = np.sum(deep_linearization_weights*last_conv_layer_output[0], axis=2)

    heatmap = np.maximum(grad_cam_map, 0)
    max_heat = np.max(heatmap)
    
    if max_heat == 0:
        max_heat = 1e-10
    heatmap /= max_heat
    
    
    return heatmap

In [ ]:
def upsample_gradCAM(activations):
    heatmap = cv2.resize(activations, (448, 448))
    heatmap /= np.sum(heatmap)

    return heatmap

In [ ]:
def explain_gradCAM(activations, img_index):
    heatmap = cv2.resize(activations, (448, 448))
    heatmap = np.expand_dims(heatmap, axis=-1)
    heatmap /= np.sum(heatmap)
    explanation = heatmap*X_expl[img_index]
    explanation -=np.min(explanation)
    explanation /= np.max(explanation)
    explanation = cv2.cvtColor(explanation, cv2.COLOR_BGR2RGB)
    #plt.matshow(explanation)
    #plt.show()  
    
    return explanation

In [ ]:
def visualize_gradCAM(activations, img_index, alpha):
    heatmap = cv2.resize(activations, (448, 448))
    heatmap = (heatmap*255).astype("uint8")
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap * alpha + X_expl[img_index]
    superimposed_img = np.clip(superimposed_img, 0, 255).astype("uint8")
    superimposed_img = cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB)

    #plt.matshow(superimposed_img)
    #plt.show()  
    
    return superimposed_img

In [ ]:
def visualize_human_attention(activations, img_index, alpha):
    #heatmap = cv2.resize(activations, (448, 448))
    heatmap = activations*(1/np.max(activations))
    heatmap = (heatmap*255).astype("uint8")
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap * alpha + X_expl[img_index]
    superimposed_img = np.clip(superimposed_img, 0, 255).astype("uint8")
    superimposed_img = cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB)

    #plt.matshow(superimposed_img)
    #plt.show()  
    
    return superimposed_img

In [ ]:
def calculate_conf_drop(model, img_index, explanation):
    y_i = model.predict(np.expand_dims(X_expl[img_index], axis=0), verbose = 0)
    pred = y_int_expl[img_index]
    y_i_c = y_i[0][pred]
    o_i_c = model.predict(np.expand_dims((explanation*255).astype("uint8"), axis=0), verbose = 0)[0][pred]
    print('yi:', y_i)
    print('oi:', model.predict(np.expand_dims((explanation*255).astype("uint8"), axis=0), verbose = 0)[0])
    print('yic:', y_i_c)
    print('oic:', o_i_c)
    conf_drop = np.maximum(0,y_i_c-o_i_c)/y_i_c
    
    return conf_drop

In [ ]:
def calculate_conf_inc(model, img_index, explanation):
    y_i = model.predict(np.expand_dims(X_expl[img_index], axis=0), verbose = 0)
    pred = y_int_expl[img_index]
    y_i_c = y_i[0][pred]
    o_i_c = model.predict(np.expand_dims((explanation*255).astype("uint8"), axis=0), verbose = 0)[0][pred]

    conf_inc = y_i_c < o_i_c

    return conf_inc

In [ ]:
def calculate_conf_win(model, img_index, explanation_pp, explanation):
    y_i = model.predict(np.expand_dims(X_expl[img_index], axis=0), verbose = 0)
    pred = y_int_expl[img_index]
    y_i_c = y_i[0][pred]
    
    o_i_c_pp = model.predict(np.expand_dims((explanation_pp*255).astype("uint8"), axis=0), verbose = 0)[0][pred]
    o_i_c = model.predict(np.expand_dims((explanation*255).astype("uint8"), axis=0), verbose = 0)[0][pred]
    
    conf_drop_pp = np.maximum(0,y_i_c-o_i_c_pp)
    conf_drop = np.maximum(0,y_i_c-o_i_c)

    return conf_drop_pp<conf_drop, conf_drop_pp>conf_drop

In [ ]:
def plot_attention_map(attention_map):
    # Define custom colormap colors
    #colors = [(0, 'white'), (0.33, 'lightgreen'), (0.66, 'yellow'), (1, 'red')]
    # Create the colormap
    #eyetracking_cmap = LinearSegmentedColormap.from_list("eyetracking_heatmap", colors)
    # Visualize with the custom colormap
    plt.imshow(attention_map)#, cmap=eyetracking_cmap, interpolation='nearest')
    plt.colorbar()
    #plt.title('Eyetracking Heatmap')
    plt.show()

## Create GradCAM ++ and GradCAM activations for each image (14x14) and save locally as *stimulus name*_GradCAMpp.npy or *stimulus name*_GradCAM.npy

In [ ]:
filepath = 'XXX'

for i in range(X_expl.shape[0]):
    gradCAM_pp_activations = gradCAMplusplus(img_array = np.expand_dims(X_expl[i], axis=0), model = model, grad_conv_model = grad_model, classifier_layer_names= classifier_layer_names)
    stimulus_name, extension = os.path.splitext(image_names_expl[i])
    filename = filepath+'/'+stimulus_name+'_GradCAMpp.npy'
    print(filename)
    np.save(filename, gradCAM_pp_activations)
    gradCAM_activations = gradCAM(img_array = np.expand_dims(X_expl[i], axis=0), model = model, grad_conv_model = grad_model, classifier_layer_names= classifier_layer_names)
    filename = filepath+'/'+stimulus_name+'_GradCAM.npy'
    print(filename)
    np.save(filename, gradCAM_activations)

## Create dataframe for each heatmap comparison (human vs. ai) using similarity measures and other features

In [ ]:
filepath_GradCAM = 'XXX'
filepath_Human_Heatmaps = 'XXX'
filepath_StudyTimelines = 'XXX'
filepath_Umfrage = 'XXX'

umfrage = pd.read_excel(filepath_Umfrage, header = 1)
umfrage = umfrage.iloc[:,6:-16]
umfrage = umfrage.drop(columns=['Teilnehmende Person: [Keine Beschreibung] 04'
                                , 'Teilnehmende Person: Ausweichoption (negativ) oder Anzahl ausgewählter Optionen'
                                , 'Datenschutz'
                                , 'Staatsangehörigkeit: Ausweichoption (negativ) oder Anzahl ausgewählter Optionen'])

umfrage['Teilnehmende Person: Student/Studentin'] = umfrage['Teilnehmende Person: Student/Studentin'].replace({'ausgewählt': 1, 'nicht gewählt': 0})
umfrage['Teilnehmende Person: Mitarbeiter/Mitarbeiterin'] = umfrage['Teilnehmende Person: Mitarbeiter/Mitarbeiterin'].replace({'ausgewählt': 1, 'nicht gewählt': 0})
umfrage['Teilnehmende Person: Extern'] = umfrage['Teilnehmende Person: Extern'].replace({'ausgewählt': 1, 'nicht gewählt': 0})
umfrage['Staatsangehörigkeit: Deutsch'] = umfrage['Staatsangehörigkeit: Deutsch'].replace({'ausgewählt': 1, 'nicht gewählt': 0})
umfrage['Staatsangehörigkeit: Sonstige'] = umfrage['Staatsangehörigkeit: Sonstige'].replace({'ausgewählt': 1, 'nicht gewählt': 0})
umfrage['Berlin'] = umfrage['Berlin'].replace({'Ja': 1, 'Nein': 0})
umfrage['Hamburg'] = umfrage['Hamburg'].replace({'Ja': 1, 'Nein': 0})
umfrage['Tel Aviv'] = umfrage['Tel Aviv'].replace({'Ja': 1, 'Nein': 0})
umfrage['Jerusalem'] = umfrage['Jerusalem'].replace({'Ja': 1, 'Nein': 0})
umfrage['Sehhilfe'] = umfrage['Sehhilfe'].replace({'Nein, ich trage weder Brille noch Kontaktlinsen': 0, 'Ja, ich trage Kontaktlinsen': 1, 'Ja, ich trage eine Brille': 2})
umfrage['Augenerkrankung'] = umfrage['Augenerkrankung'].replace({'Nein': 0})
umfrage['Farbenblindheit'] = umfrage['Farbenblindheit'].replace({'Nein': 0, 'Ja': 1})


umfrage = umfrage.rename(columns={'UserID: [01]': 'user id', 'Teilnehmende Person: Student/Studentin': 'student'
                                  , 'Teilnehmende Person: Mitarbeiter/Mitarbeiterin': 'employee'
                                  , 'Teilnehmende Person: Extern': 'external'
                                  , 'Geschlecht': 'sex'
                                  , 'Geburtsjahr: [01]': 'year of birth'
                                  , 'Staatsangehörigkeit: Deutsch': 'nationality: german'
                                  , 'Staatsangehörigkeit: Sonstige': 'nationality: others'
                                  , 'Staatsangehörigkeit: Sonstige (offene Eingabe)': 'nationality: input'
                                  , 'Fähigkeiten':'expertise', 'Sehhilfe': 'visual aid'
                                  , 'Augenerkrankung': 'eye disease'
                                  , 'Bildungsabschluss': 'educational qualification'
                                  , 'Farbenblindheit': 'color blindness'})

columns = ['CID', 'user', 'user id', 'stimuli', 'image index', 'true label', 'human prediction'
           , 'human correct', 'ai prediction'
           , 'wasserstein distance' , 'pearson correlation', 'mae'
           , 'wasserstein distance ++' , 'pearson correlation ++', 'mae ++'
           , 'confidence drop', 'confidence increase', 'win'
           , 'confidence drop ++', 'confidence increase ++', 'win ++']

rows = []
CID = 0

label_map = {0: 'B', 1: 'H', 2: 'T', 3: 'J'}

    
for user in os.listdir(filepath_Human_Heatmaps):
    print('User: ',user)
    user_id = int(user.split()[-1])
    study_timeline = pd.read_excel(filepath_StudyTimelines, sheet_name=user)
        
    for stimuli in os.listdir(os.path.join(filepath_Human_Heatmaps, user)):
        if stimuli.endswith('.npy') and stimuli.startswith('img'):
            print('****************************************')
            stimulus_name, extension = os.path.splitext(stimuli)
            print('Stimuli name: ',stimulus_name)
            
            class_name_of_stimuli = stimulus_name.split('CLASS_')[1][0]
            #print('True label: ',class_name_of_stimuli)
            
            img_index = image_names_expl.index(stimulus_name+'.png')
            #print('Stimuli image ID: ', img_index)
            
            #load user's attention map for one stimuli
            human_attention = np.load(os.path.join(filepath_Human_Heatmaps, user, stimuli))
            #print('Shape of human attention: ', human_attention.shape)
            #print('Mass of human attention: ', np.sum(human_attention))
            print('Human attention map:')
            print('Mass on attention map: ',np.sum(human_attention))
            plot_attention_map(human_attention)
            #extract user's decision for this stimuli
            human_decision = study_timeline['Antwort (B,H,T,J)'][study_timeline['Bild']==stimulus_name+'.png'].values[0]
            #print('Human pred: ', human_decision)
            human_correct = study_timeline['Richtig?'][study_timeline['Bild']==stimulus_name+'.png'].values[0]
            
            #model prediction
            ai_decision = pd.Series(np.argmax(model.predict(np.expand_dims(X_expl[img_index], axis=0), verbose = 0)))
            #print(ai_decision)
            ai_decision = ai_decision.map(label_map).values[0]
            #print('AI pred: ', ai_decision)
            
            #get GradCAM activations
            gradCAM_pp_attention = np.load(filepath_GradCAM+'/'+stimulus_name+'_GradCAMpp.npy') 
            gradCAM_attention = np.load(filepath_GradCAM+'/'+stimulus_name+'_GradCAM.npy')
            
            #upsample actications to get XAI attention map
            gradCAM_pp_up = upsample_gradCAM(gradCAM_pp_attention)
            gradCAM_up = upsample_gradCAM(gradCAM_attention)
            
            print('GradCAM ++ attention map:')
            print('Mass on attention map: ',np.sum(gradCAM_pp_up))
            plot_attention_map(gradCAM_pp_up)
            print('GradCAM attention map:')
            print('Mass on attention map: ',np.sum(gradCAM_up))
            plot_attention_map(gradCAM_up)  
            
            #print('Shape of GradCAM ++ attention: ', gradCAM_pp_up.shape)
            #print('Mass of GradCAM ++ attention: ', np.sum(gradCAM_pp_up))
            #print('Max of GradCAM ++ attention: ', np.max(gradCAM_pp_up.flatten()))
            #print('Min of GradCAM ++ attention: ', np.min(gradCAM_pp_up.flatten()))
            
            #print('Shape of GradCAM attention: ', gradCAM_up.shape)
            #print('Mass of GradCAM attention: ', np.sum(gradCAM_up))    
            #print('Max of GradCAM attention: ', np.max(gradCAM_up.flatten()))
            #print('Min of GradCAM attention: ', np.min(gradCAM_up.flatten()))     
            
            wasserstein_distance_GradCAM_pp = wasserstein_distance(gradCAM_pp_up.flatten(), human_attention.flatten())
            #print(wasserstein_distance_GradCAM_pp)
            wasserstein_distance_GradCAM = wasserstein_distance(gradCAM_up.flatten(), human_attention.flatten())
            #print(wasserstein_distance_GradCAM)
            
            pearson_corr_GradCAM_pp = np.corrcoef(gradCAM_pp_up.flatten(), human_attention.flatten())[0, 1]
            #print(pearson_corr_GradCAM_pp)
            pearson_corr_GradCAM = np.corrcoef(gradCAM_up.flatten(), human_attention.flatten())[0, 1]
            #print(pearson_corr_GradCAM)
            
            MAE_GradCAM_pp = np.mean(np.abs(gradCAM_pp_up.flatten() - human_attention.flatten()))
            #print(MAE_GradCAM_pp)
            MAE_GradCAM = np.mean(np.abs(gradCAM_up.flatten() - human_attention.flatten()))
            #print(MAE_GradCAM)
            
            #explanation for model input
            print('GradCAM ++ explanation map:')
            gradCAM_pp_expl = explain_gradCAM(activations = gradCAM_pp_attention, img_index = img_index)
            print('GradCAM explanation map:')
            gradCAM_expl = explain_gradCAM(activations = gradCAM_attention, img_index = img_index)
    
            #faithfulness evaluation metrics for each image
            conf_drop_pp_i = calculate_conf_drop(model = model, img_index = img_index, explanation = gradCAM_pp_expl)
            conf_drop_i = calculate_conf_drop(model = model, img_index = img_index, explanation = gradCAM_expl)

            conf_inc_pp_i = (calculate_conf_inc(model = model, img_index = img_index, explanation = gradCAM_pp_expl)).astype(int)
            conf_inc_i = (calculate_conf_inc(model = model, img_index = img_index, explanation = gradCAM_expl)).astype(int)

            win_pp_i, win_i = calculate_conf_win(model = model, img_index = img_index, explanation_pp = gradCAM_pp_expl, explanation = gradCAM_expl)

            row_data = {'CID': CID, 'user': user, 'user id': user_id,'stimuli': stimulus_name
                           , 'image index': img_index, 'true label': class_name_of_stimuli
                           , 'human prediction': human_decision, 'human correct': human_correct
                           , 'ai prediction': ai_decision
                           , 'wasserstein distance ++': wasserstein_distance_GradCAM_pp
                           , 'pearson correlation ++': pearson_corr_GradCAM_pp
                           , 'mae ++': MAE_GradCAM_pp
                           , 'confidence drop ++': conf_drop_pp_i, 'confidence increase ++': conf_inc_pp_i
                           , 'win ++': (win_pp_i).astype(int)
                           , 'wasserstein distance': wasserstein_distance_GradCAM
                           , 'pearson correlation': pearson_corr_GradCAM
                           , 'mae': MAE_GradCAM
                           , 'confidence drop': conf_drop_i, 'confidence increase': conf_inc_i
                           , 'win': (win_i).astype(int)}
            
            rows.append(row_data)
            
            CID += 1     
    
df = pd.DataFrame(rows, columns=columns)
df['ai correct'] = (df['true label'] == df['ai prediction']).astype(int)
merged_df = pd.merge(df, umfrage, on='user id', how='left')

merged_df = pd.merge(merged_df, merged_df.groupby('stimuli')['human correct'].mean().reset_index(), on = 'stimuli')
merged_df = merged_df.rename(columns={'human correct_y': 'average human image accuracy'})
merged_df = merged_df.rename(columns={'human correct_x': 'human correct'})

merged_df['both german cities visited'] = (merged_df['Berlin'] & merged_df['Hamburg'])
merged_df['both israel cities visited'] = (merged_df['Tel Aviv'] & merged_df['Jerusalem'])

merged_df = pd.merge(merged_df, merged_df.groupby('user')['human correct'].mean().reset_index(), on = 'user')
merged_df = merged_df.rename(columns={'human correct_y': 'human accuracy'})
merged_df = merged_df.rename(columns={'human correct_x': 'human correct'})

In [ ]:
filepath_GradCAM = 'XXX'
filepath_Human_Heatmaps = 'XXX'
filepath_StudyTimelines = 'XXX'
filepath_Umfrage = 'XXX'

heatmap_name_pp_1 = 'img_31.77692456977874,35.20849815507335CLASS_Jerusalem11_GradCAMpp'
heatmap_name_1 = 'img_31.77692456977874,35.20849815507335CLASS_Jerusalem11_GradCAM'
heatmap_name_pp_2 = 'img_32.0574547334093,34.765539699605604CLASS_Tel-Aviv7_GradCAMpp'
heatmap_name_2 = 'img_32.0574547334093,34.765539699605604CLASS_Tel-Aviv7_GradCAM'
heatmap_name_pp_3 = 'img_52.49465572860476,13.420295431164686CLASS_Berlin10_GradCAMpp'
heatmap_name_3 = 'img_52.49465572860476,13.420295431164686CLASS_Berlin10_GradCAM'
heatmap_name_pp_4 = 'img_53.548836682781165,9.981510481779921CLASS_Hamburg24_GradCAMpp'
heatmap_name_4 = 'img_53.548836682781165,9.981510481779921CLASS_Hamburg24_GradCAM'

img_index_1 = image_names_expl.index('img_31.77692456977874,35.20849815507335CLASS_Jerusalem11.png')
img_index_2 = image_names_expl.index('img_32.0574547334093,34.765539699605604CLASS_Tel-Aviv7.png')
img_index_3 = image_names_expl.index('img_52.49465572860476,13.420295431164686CLASS_Berlin10.png')
img_index_4 = image_names_expl.index('img_53.548836682781165,9.981510481779921CLASS_Hamburg24.png')

label_map = {0: 'B', 1: 'H', 2: 'T', 3: 'J'}

#ai_decision = pd.Series(np.argmax(model.predict(np.expand_dims(X_expl[img_index], axis=0), verbose = 0)))
#ai_decision = ai_decision.map(label_map).values[0]
#print('AI pred: ', ai_decision)
            
gradCAM_pp_attention_1 = np.load(filepath_GradCAM+'/'+heatmap_name_pp_1+'.npy') 
gradCAM_attention_1 = np.load(filepath_GradCAM+'/'+heatmap_name_1+'.npy')
human_attention_1 = np.load(filepath_Human_Heatmaps+'/User ID 2/'+'img_31.77692456977874,35.20849815507335CLASS_Jerusalem11'+'.npy') 

gradCAM_pp_attention_2 = np.load(filepath_GradCAM+'/'+heatmap_name_pp_2+'.npy') 
gradCAM_attention_2 = np.load(filepath_GradCAM+'/'+heatmap_name_2+'.npy')
human_attention_2 = np.load(filepath_Human_Heatmaps+'/User ID 6/'+'img_32.0574547334093,34.765539699605604CLASS_Tel-Aviv7'+'.npy') 

gradCAM_pp_attention_3 = np.load(filepath_GradCAM+'/'+heatmap_name_pp_3+'.npy') 
gradCAM_attention_3 = np.load(filepath_GradCAM+'/'+heatmap_name_3+'.npy')
human_attention_3 = np.load(filepath_Human_Heatmaps+'/User ID 6/'+'img_52.49465572860476,13.420295431164686CLASS_Berlin10'+'.npy') 

gradCAM_pp_attention_4 = np.load(filepath_GradCAM+'/'+heatmap_name_pp_4+'.npy') 
gradCAM_attention_4 = np.load(filepath_GradCAM+'/'+heatmap_name_4+'.npy')
human_attention_4 = np.load(filepath_Human_Heatmaps+'/User ID 32/'+'img_53.548836682781165,9.981510481779921CLASS_Hamburg24'+'.npy') 
     
gradCAM_pp_up_1 = upsample_gradCAM(gradCAM_pp_attention_1)
gradCAM_up_1 = upsample_gradCAM(gradCAM_attention_1)
gradCAM_pp_up_2 = upsample_gradCAM(gradCAM_pp_attention_2)
gradCAM_up_2 = upsample_gradCAM(gradCAM_attention_2)
gradCAM_pp_up_3 = upsample_gradCAM(gradCAM_pp_attention_3)
gradCAM_up_3 = upsample_gradCAM(gradCAM_attention_3)
gradCAM_pp_up_4 = upsample_gradCAM(gradCAM_pp_attention_4)
gradCAM_up_4 = upsample_gradCAM(gradCAM_attention_4)

original_img_1 = np.clip(X_expl[img_index_1], 0, 255).astype("uint8")
original_img_1 = cv2.cvtColor(original_img_1, cv2.COLOR_BGR2RGB)
gradCAMpp_1 = visualize_gradCAM(gradCAM_pp_attention_1, img_index_1, alpha = 0.4)
gradCAM_1 = visualize_gradCAM(gradCAM_attention_1, img_index_1, alpha = 0.4)
human_attention_1 = visualize_human_attention(human_attention_1, img_index_1, alpha = 0.4)

original_img_2 = np.clip(X_expl[img_index_2], 0, 255).astype("uint8")
original_img_2 = cv2.cvtColor(original_img_2, cv2.COLOR_BGR2RGB)
gradCAMpp_2 = visualize_gradCAM(gradCAM_pp_attention_2, img_index_2, alpha = 0.4)
gradCAM_2 = visualize_gradCAM(gradCAM_attention_2, img_index_2, alpha = 0.4)
human_attention_2 = visualize_human_attention(human_attention_2, img_index_2, alpha = 0.4)

original_img_3 = np.clip(X_expl[img_index_3], 0, 255).astype("uint8")
original_img_3 = cv2.cvtColor(original_img_3, cv2.COLOR_BGR2RGB)
gradCAMpp_3 = visualize_gradCAM(gradCAM_pp_attention_3, img_index_3, alpha = 0.4)
gradCAM_3 = visualize_gradCAM(gradCAM_attention_3, img_index_3, alpha = 0.4)
human_attention_3 = visualize_human_attention(human_attention_3, img_index_3, alpha = 0.4)

original_img_4 = np.clip(X_expl[img_index_4], 0, 255).astype("uint8")
original_img_4 = cv2.cvtColor(original_img_4, cv2.COLOR_BGR2RGB)
gradCAMpp_4 = visualize_gradCAM(gradCAM_pp_attention_4, img_index_4, alpha = 0.4)
gradCAM_4 = visualize_gradCAM(gradCAM_attention_4, img_index_4, alpha = 0.4)
human_attention_4 = visualize_human_attention(human_attention_4, img_index_4, alpha = 0.4)

fig, axes = plt.subplots(4, 4, figsize=(12, 12))

axes[0, 0].imshow(original_img_1)
axes[0, 0].set_title('original image (Jerusalem)')

axes[1, 0].imshow(gradCAMpp_1)
axes[1, 0].set_title('Grad-CAM++ attention map')

axes[2, 0].imshow(gradCAM_1)
axes[2, 0].set_title('Grad-CAM attention map')

axes[3, 0].imshow(human_attention_1)
axes[3, 0].set_title('human attention map')

axes[0, 1].imshow(original_img_2)
axes[0, 1].set_title('original image (Tel Aviv)')

axes[1, 1].imshow(gradCAMpp_2)
axes[1, 1].set_title('Grad-CAM++ attention map')

axes[2, 1].imshow(gradCAM_2)
axes[2, 1].set_title('Grad-CAM attention map')

axes[3, 1].imshow(human_attention_2)
axes[3, 1].set_title('human attention map')

axes[0, 2].imshow(original_img_3)
axes[0, 2].set_title('original image (Berlin)')

axes[1, 2].imshow(gradCAMpp_3)
axes[1, 2].set_title('Grad-CAM++ attention map')

axes[2, 2].imshow(gradCAM_3)
axes[2, 2].set_title('Grad-CAM attention map')

axes[3, 2].imshow(human_attention_3)
axes[3, 2].set_title('human attention map')

axes[0, 3].imshow(original_img_4)
axes[0, 3].set_title('original image (Hamburg)')

axes[1, 3].imshow(gradCAMpp_4)
axes[1, 3].set_title('Grad-CAM++ attention map')

axes[2, 3].imshow(gradCAM_4)
axes[2, 3].set_title('Grad-CAM attention map')

axes[3, 3].imshow(human_attention_4)
axes[3, 3].set_title('human attention map')

for ax_row in axes:
    for ax in ax_row:
        ax.axis('off')
        
plt.suptitle('Examples of AI and human attention maps for each class', fontsize=13)   

plt.tight_layout()
plt.savefig('attentionmaps.png', dpi=300)


In [ ]:
filepath_GradCAM = 'XXX'
filepath_Human_Heatmaps = 'XXX'
filepath_StudyTimelines = 'XXX'
filepath_Umfrage = 'XXX'

heatmap_name_pp_1 = 'img_31.768322995172117,35.18716699870462CLASS_Jerusalem21_GradCAMpp'
heatmap_name_1 = 'img_31.768322995172117,35.18716699870462CLASS_Jerusalem21_GradCAM'
heatmap_name_pp_2 = 'img_32.056770212808196,34.7732904745884CLASS_Tel-Aviv25_GradCAMpp'
heatmap_name_2 = 'img_32.056770212808196,34.7732904745884CLASS_Tel-Aviv25_GradCAM'
heatmap_name_pp_3 = 'img_52.50301222133445,13.328699938075516CLASS_Berlin16_GradCAMpp'
heatmap_name_3 = 'img_52.50301222133445,13.328699938075516CLASS_Berlin16_GradCAM'
heatmap_name_pp_4 = 'img_53.59310855742811,10.002262467095225CLASS_Hamburg20_GradCAMpp'
heatmap_name_4 = 'img_53.59310855742811,10.002262467095225CLASS_Hamburg20_GradCAM'

img_index_1 = image_names_expl.index('img_31.768322995172117,35.18716699870462CLASS_Jerusalem21.png')
img_index_2 = image_names_expl.index('img_32.056770212808196,34.7732904745884CLASS_Tel-Aviv25.png')
img_index_3 = image_names_expl.index('img_52.50301222133445,13.328699938075516CLASS_Berlin16.png')
img_index_4 = image_names_expl.index('img_53.59310855742811,10.002262467095225CLASS_Hamburg20.png')

label_map = {0: 'B', 1: 'H', 2: 'T', 3: 'J'}

#ai_decision = pd.Series(np.argmax(model.predict(np.expand_dims(X_expl[img_index], axis=0), verbose = 0)))
#ai_decision = ai_decision.map(label_map).values[0]
#print('AI pred: ', ai_decision)
            
gradCAM_pp_attention_1 = np.load(filepath_GradCAM+'/'+heatmap_name_pp_1+'.npy') 
gradCAM_attention_1 = np.load(filepath_GradCAM+'/'+heatmap_name_1+'.npy')
human_attention_1 = np.load(filepath_Human_Heatmaps+'/User ID 32/'+'img_31.768322995172117,35.18716699870462CLASS_Jerusalem21'+'.npy') 

gradCAM_pp_attention_2 = np.load(filepath_GradCAM+'/'+heatmap_name_pp_2+'.npy') 
gradCAM_attention_2 = np.load(filepath_GradCAM+'/'+heatmap_name_2+'.npy')
human_attention_2 = np.load(filepath_Human_Heatmaps+'/User ID 19/'+'img_32.056770212808196,34.7732904745884CLASS_Tel-Aviv25'+'.npy') 

gradCAM_pp_attention_3 = np.load(filepath_GradCAM+'/'+heatmap_name_pp_3+'.npy') 
gradCAM_attention_3 = np.load(filepath_GradCAM+'/'+heatmap_name_3+'.npy')
human_attention_3 = np.load(filepath_Human_Heatmaps+'/User ID 31/'+'img_52.50301222133445,13.328699938075516CLASS_Berlin16'+'.npy') 

gradCAM_pp_attention_4 = np.load(filepath_GradCAM+'/'+heatmap_name_pp_4+'.npy') 
gradCAM_attention_4 = np.load(filepath_GradCAM+'/'+heatmap_name_4+'.npy')
human_attention_4 = np.load(filepath_Human_Heatmaps+'/User ID 41/'+'img_53.59310855742811,10.002262467095225CLASS_Hamburg20'+'.npy') 
     
gradCAM_pp_up_1 = upsample_gradCAM(gradCAM_pp_attention_1)
gradCAM_up_1 = upsample_gradCAM(gradCAM_attention_1)
gradCAM_pp_up_2 = upsample_gradCAM(gradCAM_pp_attention_2)
gradCAM_up_2 = upsample_gradCAM(gradCAM_attention_2)
gradCAM_pp_up_3 = upsample_gradCAM(gradCAM_pp_attention_3)
gradCAM_up_3 = upsample_gradCAM(gradCAM_attention_3)
gradCAM_pp_up_4 = upsample_gradCAM(gradCAM_pp_attention_4)
gradCAM_up_4 = upsample_gradCAM(gradCAM_attention_4)

original_img_1 = np.clip(X_expl[img_index_1], 0, 255).astype("uint8")
original_img_1 = cv2.cvtColor(original_img_1, cv2.COLOR_BGR2RGB)
gradCAMpp_1 = visualize_gradCAM(gradCAM_pp_attention_1, img_index_1, alpha = 0.4)
gradCAM_1 = visualize_gradCAM(gradCAM_attention_1, img_index_1, alpha = 0.4)
human_attention_1 = visualize_human_attention(human_attention_1, img_index_1, alpha = 0.4)

original_img_2 = np.clip(X_expl[img_index_2], 0, 255).astype("uint8")
original_img_2 = cv2.cvtColor(original_img_2, cv2.COLOR_BGR2RGB)
gradCAMpp_2 = visualize_gradCAM(gradCAM_pp_attention_2, img_index_2, alpha = 0.4)
gradCAM_2 = visualize_gradCAM(gradCAM_attention_2, img_index_2, alpha = 0.4)
human_attention_2 = visualize_human_attention(human_attention_2, img_index_2, alpha = 0.4)

original_img_3 = np.clip(X_expl[img_index_3], 0, 255).astype("uint8")
original_img_3 = cv2.cvtColor(original_img_3, cv2.COLOR_BGR2RGB)
gradCAMpp_3 = visualize_gradCAM(gradCAM_pp_attention_3, img_index_3, alpha = 0.4)
gradCAM_3 = visualize_gradCAM(gradCAM_attention_3, img_index_3, alpha = 0.4)
human_attention_3 = visualize_human_attention(human_attention_3, img_index_3, alpha = 0.4)

original_img_4 = np.clip(X_expl[img_index_4], 0, 255).astype("uint8")
original_img_4 = cv2.cvtColor(original_img_4, cv2.COLOR_BGR2RGB)
gradCAMpp_4 = visualize_gradCAM(gradCAM_pp_attention_4, img_index_4, alpha = 0.4)
gradCAM_4 = visualize_gradCAM(gradCAM_attention_4, img_index_4, alpha = 0.4)
human_attention_4 = visualize_human_attention(human_attention_4, img_index_4, alpha = 0.4)

fig, axes = plt.subplots(4, 4, figsize=(12, 12))

axes[0, 0].imshow(original_img_1)
axes[0, 0].set_title('original Image (Jerusalem)')

axes[1, 0].imshow(gradCAMpp_1)
axes[1, 0].set_title('Grad-CAM++ attention map')

axes[2, 0].imshow(gradCAM_1)
axes[2, 0].set_title('Grad-CAM attention map')

axes[3, 0].imshow(human_attention_1)
axes[3, 0].set_title('human attention map')

axes[0, 1].imshow(original_img_2)
axes[0, 1].set_title('original Image (Tel Aviv)')

axes[1, 1].imshow(gradCAMpp_2)
axes[1, 1].set_title('Grad-CAM++ attention map')

axes[2, 1].imshow(gradCAM_2)
axes[2, 1].set_title('Grad-CAM attention map')

axes[3, 1].imshow(human_attention_2)
axes[3, 1].set_title('human attention map')

axes[0, 2].imshow(original_img_3)
axes[0, 2].set_title('original Image (Berlin)')

axes[1, 2].imshow(gradCAMpp_3)
axes[1, 2].set_title('Grad-CAM++ attention map')

axes[2, 2].imshow(gradCAM_3)
axes[2, 2].set_title('Grad-CAM attention map')

axes[3, 2].imshow(human_attention_3)
axes[3, 2].set_title('human attention map')

axes[0, 3].imshow(original_img_4)
axes[0, 3].set_title('original Image (Hamburg)')

axes[1, 3].imshow(gradCAMpp_4)
axes[1, 3].set_title('Grad-CAM++ attention map')

axes[2, 3].imshow(gradCAM_4)
axes[2, 3].set_title('Grad-CAM attention map')

axes[3, 3].imshow(human_attention_4)
axes[3, 3].set_title('human attention map')

for ax_row in axes:
    for ax in ax_row:
        ax.axis('off')
        
plt.tight_layout()

plt.savefig('attentionmaps2.png', dpi=300)


In [ ]:
filepath_GradCAM = 'XXX'
filepath_Human_Heatmaps = 'XXX'
filepath_StudyTimelines = 'XXX'
filepath_Umfrage = 'XXX'

heatmap_name_pp_1 = 'img_31.763214851900877,35.20885600974014CLASS_Jerusalem17_GradCAMpp'
heatmap_name_1 = 'img_31.763214851900877,35.20885600974014CLASS_Jerusalem17_GradCAM'
heatmap_name_pp_2 = 'img_32.0840529613404,34.78256479352893CLASS_Tel-Aviv10_GradCAMpp'
heatmap_name_2 = 'img_32.0840529613404,34.78256479352893CLASS_Tel-Aviv10_GradCAM'
heatmap_name_pp_3 = 'img_52.52473202450015,13.348054821127073CLASS_Berlin22_GradCAMpp'
heatmap_name_3 = 'img_52.52473202450015,13.348054821127073CLASS_Berlin22_GradCAM'
heatmap_name_pp_4 = 'img_53.548502941071426,10.003652538092444CLASS_Hamburg23_GradCAMpp'
heatmap_name_4 = 'img_53.548502941071426,10.003652538092444CLASS_Hamburg23_GradCAM'

img_index_1 = image_names_expl.index('img_31.763214851900877,35.20885600974014CLASS_Jerusalem17.png')
img_index_2 = image_names_expl.index('img_32.0840529613404,34.78256479352893CLASS_Tel-Aviv10.png')
img_index_3 = image_names_expl.index('img_52.52473202450015,13.348054821127073CLASS_Berlin22.png')
img_index_4 = image_names_expl.index('img_53.548502941071426,10.003652538092444CLASS_Hamburg23.png')

label_map = {0: 'B', 1: 'H', 2: 'T', 3: 'J'}

#ai_decision = pd.Series(np.argmax(model.predict(np.expand_dims(X_expl[img_index], axis=0), verbose = 0)))
#ai_decision = ai_decision.map(label_map).values[0]
#print('AI pred: ', ai_decision)
            
gradCAM_pp_attention_1 = np.load(filepath_GradCAM+'/'+heatmap_name_pp_1+'.npy') 
gradCAM_attention_1 = np.load(filepath_GradCAM+'/'+heatmap_name_1+'.npy')
human_attention_1 = np.load(filepath_Human_Heatmaps+'/User ID 28/'+'img_31.763214851900877,35.20885600974014CLASS_Jerusalem17'+'.npy') 

gradCAM_pp_attention_2 = np.load(filepath_GradCAM+'/'+heatmap_name_pp_2+'.npy') 
gradCAM_attention_2 = np.load(filepath_GradCAM+'/'+heatmap_name_2+'.npy')
human_attention_2 = np.load(filepath_Human_Heatmaps+'/User ID 21/'+'img_32.0840529613404,34.78256479352893CLASS_Tel-Aviv10'+'.npy') 

gradCAM_pp_attention_3 = np.load(filepath_GradCAM+'/'+heatmap_name_pp_3+'.npy') 
gradCAM_attention_3 = np.load(filepath_GradCAM+'/'+heatmap_name_3+'.npy')
human_attention_3 = np.load(filepath_Human_Heatmaps+'/User ID 26/'+'img_52.52473202450015,13.348054821127073CLASS_Berlin22'+'.npy') 

gradCAM_pp_attention_4 = np.load(filepath_GradCAM+'/'+heatmap_name_pp_4+'.npy') 
gradCAM_attention_4 = np.load(filepath_GradCAM+'/'+heatmap_name_4+'.npy')
human_attention_4 = np.load(filepath_Human_Heatmaps+'/User ID 44/'+'img_53.548502941071426,10.003652538092444CLASS_Hamburg23'+'.npy') 
     
gradCAM_pp_up_1 = upsample_gradCAM(gradCAM_pp_attention_1)
gradCAM_up_1 = upsample_gradCAM(gradCAM_attention_1)
gradCAM_pp_up_2 = upsample_gradCAM(gradCAM_pp_attention_2)
gradCAM_up_2 = upsample_gradCAM(gradCAM_attention_2)
gradCAM_pp_up_3 = upsample_gradCAM(gradCAM_pp_attention_3)
gradCAM_up_3 = upsample_gradCAM(gradCAM_attention_3)
gradCAM_pp_up_4 = upsample_gradCAM(gradCAM_pp_attention_4)
gradCAM_up_4 = upsample_gradCAM(gradCAM_attention_4)

original_img_1 = np.clip(X_expl[img_index_1], 0, 255).astype("uint8")
original_img_1 = cv2.cvtColor(original_img_1, cv2.COLOR_BGR2RGB)
gradCAMpp_1 = visualize_gradCAM(gradCAM_pp_attention_1, img_index_1, alpha = 0.4)
gradCAM_1 = visualize_gradCAM(gradCAM_attention_1, img_index_1, alpha = 0.4)
human_attention_1 = visualize_human_attention(human_attention_1, img_index_1, alpha = 0.4)

original_img_2 = np.clip(X_expl[img_index_2], 0, 255).astype("uint8")
original_img_2 = cv2.cvtColor(original_img_2, cv2.COLOR_BGR2RGB)
gradCAMpp_2 = visualize_gradCAM(gradCAM_pp_attention_2, img_index_2, alpha = 0.4)
gradCAM_2 = visualize_gradCAM(gradCAM_attention_2, img_index_2, alpha = 0.4)
human_attention_2 = visualize_human_attention(human_attention_2, img_index_2, alpha = 0.4)

original_img_3 = np.clip(X_expl[img_index_3], 0, 255).astype("uint8")
original_img_3 = cv2.cvtColor(original_img_3, cv2.COLOR_BGR2RGB)
gradCAMpp_3 = visualize_gradCAM(gradCAM_pp_attention_3, img_index_3, alpha = 0.4)
gradCAM_3 = visualize_gradCAM(gradCAM_attention_3, img_index_3, alpha = 0.4)
human_attention_3 = visualize_human_attention(human_attention_3, img_index_3, alpha = 0.4)

original_img_4 = np.clip(X_expl[img_index_4], 0, 255).astype("uint8")
original_img_4 = cv2.cvtColor(original_img_4, cv2.COLOR_BGR2RGB)
gradCAMpp_4 = visualize_gradCAM(gradCAM_pp_attention_4, img_index_4, alpha = 0.4)
gradCAM_4 = visualize_gradCAM(gradCAM_attention_4, img_index_4, alpha = 0.4)
human_attention_4 = visualize_human_attention(human_attention_4, img_index_4, alpha = 0.4)

fig, axes = plt.subplots(4, 4, figsize=(12, 12))

axes[0, 0].imshow(original_img_1)
axes[0, 0].set_title('original Image (Jerusalem)')

axes[1, 0].imshow(gradCAMpp_1)
axes[1, 0].set_title('Grad-CAM++ attention map')

axes[2, 0].imshow(gradCAM_1)
axes[2, 0].set_title('Grad-CAM attention map')

axes[3, 0].imshow(human_attention_1)
axes[3, 0].set_title('human attention map')

axes[0, 1].imshow(original_img_2)
axes[0, 1].set_title('original Image (Tel Aviv)')

axes[1, 1].imshow(gradCAMpp_2)
axes[1, 1].set_title('Grad-CAM++ attention map')

axes[2, 1].imshow(gradCAM_2)
axes[2, 1].set_title('Grad-CAM attention map')

axes[3, 1].imshow(human_attention_2)
axes[3, 1].set_title('human attention map')

axes[0, 2].imshow(original_img_3)
axes[0, 2].set_title('original Image (Berlin)')

axes[1, 2].imshow(gradCAMpp_3)
axes[1, 2].set_title('Grad-CAM++ attention map')

axes[2, 2].imshow(gradCAM_3)
axes[2, 2].set_title('Grad-CAM attention map')

axes[3, 2].imshow(human_attention_3)
axes[3, 2].set_title('human attention map')

axes[0, 3].imshow(original_img_4)
axes[0, 3].set_title('original Image (Hamburg)')

axes[1, 3].imshow(gradCAMpp_4)
axes[1, 3].set_title('Grad-CAM++ attention map')

axes[2, 3].imshow(gradCAM_4)
axes[2, 3].set_title('Grad-CAM attention map')

axes[3, 3].imshow(human_attention_4)
axes[3, 3].set_title('human attention map')

for ax_row in axes:
    for ax in ax_row:
        ax.axis('off')
        
plt.tight_layout()

plt.savefig('attentionmaps3.png', dpi=300)


In [ ]:
filepath_GradCAM = 'XXX'
filepath_Human_Heatmaps = 'XXX'
filepath_StudyTimelines = 'XXX'
filepath_Umfrage = 'XXX'

heatmap_name_pp_1 = 'img_31.77692456977874,35.20849815507335CLASS_Jerusalem11_GradCAMpp'
heatmap_name_1 = 'img_31.77692456977874,35.20849815507335CLASS_Jerusalem11_GradCAM'
heatmap_name_pp_2 = 'img_32.0574547334093,34.765539699605604CLASS_Tel-Aviv7_GradCAMpp'
heatmap_name_2 = 'img_32.0574547334093,34.765539699605604CLASS_Tel-Aviv7_GradCAM'
heatmap_name_pp_3 = 'img_52.49465572860476,13.420295431164686CLASS_Berlin10_GradCAMpp'
heatmap_name_3 = 'img_52.49465572860476,13.420295431164686CLASS_Berlin10_GradCAM'
heatmap_name_pp_4 = 'img_53.548836682781165,9.981510481779921CLASS_Hamburg24_GradCAMpp'
heatmap_name_4 = 'img_53.548836682781165,9.981510481779921CLASS_Hamburg24_GradCAM'

img_index_1 = image_names_expl.index('img_31.77692456977874,35.20849815507335CLASS_Jerusalem11.png')
img_index_2 = image_names_expl.index('img_32.0574547334093,34.765539699605604CLASS_Tel-Aviv7.png')
img_index_3 = image_names_expl.index('img_52.49465572860476,13.420295431164686CLASS_Berlin10.png')
img_index_4 = image_names_expl.index('img_53.548836682781165,9.981510481779921CLASS_Hamburg24.png')

label_map = {0: 'B', 1: 'H', 2: 'T', 3: 'J'}

#ai_decision = pd.Series(np.argmax(model.predict(np.expand_dims(X_expl[img_index], axis=0), verbose = 0)))
#ai_decision = ai_decision.map(label_map).values[0]
#print('AI pred: ', ai_decision)
            
gradCAM_pp_attention_1 = np.load(filepath_GradCAM+'/'+heatmap_name_pp_1+'.npy') 
gradCAM_attention_1 = np.load(filepath_GradCAM+'/'+heatmap_name_1+'.npy')

gradCAM_pp_attention_2 = np.load(filepath_GradCAM+'/'+heatmap_name_pp_2+'.npy') 
gradCAM_attention_2 = np.load(filepath_GradCAM+'/'+heatmap_name_2+'.npy')


gradCAM_pp_attention_3 = np.load(filepath_GradCAM+'/'+heatmap_name_pp_3+'.npy') 
gradCAM_attention_3 = np.load(filepath_GradCAM+'/'+heatmap_name_3+'.npy')


gradCAM_pp_attention_4 = np.load(filepath_GradCAM+'/'+heatmap_name_pp_4+'.npy') 
gradCAM_attention_4 = np.load(filepath_GradCAM+'/'+heatmap_name_4+'.npy')

     
gradCAM_pp_expl_1 = explain_gradCAM(activations = gradCAM_pp_attention_1, img_index = img_index_1)
gradCAM_expl_1 = explain_gradCAM(activations = gradCAM_attention_1, img_index = img_index_1)

gradCAM_pp_expl_2 = explain_gradCAM(activations = gradCAM_pp_attention_2, img_index = img_index_2)
gradCAM_expl_2 = explain_gradCAM(activations = gradCAM_attention_2, img_index = img_index_2)

gradCAM_pp_expl_3 = explain_gradCAM(activations = gradCAM_pp_attention_3, img_index = img_index_3)
gradCAM_expl_3 = explain_gradCAM(activations = gradCAM_attention_3, img_index = img_index_3)

gradCAM_pp_expl_4 = explain_gradCAM(activations = gradCAM_pp_attention_4, img_index = img_index_4)
gradCAM_expl_4 = explain_gradCAM(activations = gradCAM_attention_4, img_index = img_index_4)

original_img_1 = np.clip(X_expl[img_index_1], 0, 255).astype("uint8")
original_img_1 = cv2.cvtColor(original_img_1, cv2.COLOR_BGR2RGB)

original_img_2 = np.clip(X_expl[img_index_2], 0, 255).astype("uint8")
original_img_2 = cv2.cvtColor(original_img_2, cv2.COLOR_BGR2RGB)

original_img_3 = np.clip(X_expl[img_index_3], 0, 255).astype("uint8")
original_img_3 = cv2.cvtColor(original_img_3, cv2.COLOR_BGR2RGB)

original_img_4 = np.clip(X_expl[img_index_4], 0, 255).astype("uint8")
original_img_4 = cv2.cvtColor(original_img_4, cv2.COLOR_BGR2RGB)

fig, axes = plt.subplots(3, 4, figsize=(12, 9))

axes[0, 0].imshow(original_img_1)
axes[0, 0].set_title('original image (Jerusalem)')

axes[1, 0].imshow(gradCAM_pp_expl_1)
axes[1, 0].set_title('Grad-CAM++ explanation map')

axes[2, 0].imshow(gradCAM_expl_1)
axes[2, 0].set_title('Grad-CAM explanation map')

axes[0, 1].imshow(original_img_2)
axes[0, 1].set_title('original image (Tel Aviv)')

axes[1, 1].imshow(gradCAM_pp_expl_2)
axes[1, 1].set_title('Grad-CAM++ explanation map')

axes[2, 1].imshow(gradCAM_expl_2)
axes[2, 1].set_title('Grad-CAM explanation map')

axes[0, 2].imshow(original_img_3)
axes[0, 2].set_title('original image (Berlin)')

axes[1, 2].imshow(gradCAM_pp_expl_3)
axes[1, 2].set_title('Grad-CAM++ explanation map')

axes[2, 2].imshow(gradCAM_expl_3)
axes[2, 2].set_title('Grad-CAM explanation map')

axes[0, 3].imshow(original_img_4)
axes[0, 3].set_title('original image (Hamburg)')

axes[1, 3].imshow(gradCAM_pp_expl_4)
axes[1, 3].set_title('Grad-CAM++ explanation map')

axes[2, 3].imshow(gradCAM_expl_4)
axes[2, 3].set_title('Grad-CAM explanation map')

for ax_row in axes:
    for ax in ax_row:
        ax.axis('off')

plt.suptitle('Examples of AI explanation maps for each class', fontsize=13)
plt.tight_layout()

plt.savefig('explanationmaps.png', dpi=300)



In [ ]:
conf_drop_pp = 0
conf_drop = 0

conf_inc_pp = 0
conf_inc = 0

win_pp = 0
win = 0

for i in range(X_expl.shape[0]):
    
    gradCAM_pp_activations = gradCAMplusplus(img_array = np.expand_dims(X_expl[i], axis=0), model = model, grad_conv_model = grad_model, classifier_layer_names= classifier_layer_names)
    gradCAM_activations = gradCAM(img_array = np.expand_dims(X_expl[i], axis=0), model = model, grad_conv_model = grad_model, classifier_layer_names= classifier_layer_names)
    
    print('************************************')
    print('Grad-CAM ++ Input for evaluation:')
    gradCAM_pp_expl = explain_gradCAM(activations = gradCAM_pp_activations, img_index = i)
    print('Grad-CAM Input for evaluation:')
    gradCAM_expl = explain_gradCAM(activations = gradCAM_activations, img_index = i)
    
    conf_drop_pp_i = calculate_conf_drop(model = model, img_index = i, explanation = gradCAM_pp_expl)
    conf_drop_pp += conf_drop_pp_i

    conf_drop_i = calculate_conf_drop(model = model, img_index = i, explanation = gradCAM_expl)
    conf_drop += conf_drop_i
    
    conf_inc_pp_i = calculate_conf_inc(model = model, img_index = i, explanation = gradCAM_pp_expl)
    conf_inc_pp += conf_inc_pp_i

    conf_inc_i = calculate_conf_inc(model = model, img_index = i, explanation = gradCAM_expl)
    conf_inc += conf_inc_i
    
    win_pp_i, win_i = calculate_conf_win(model = model, img_index = i, explanation_pp = gradCAM_pp_expl, explanation = gradCAM_expl)
    win_pp += win_pp_i
    win += win_i
    
avg_conf_drop_pp = conf_drop_pp*100/(X_expl.shape[0])
avg_conf_drop = conf_drop*100/(X_expl.shape[0])

avg_conf_inc_pp = conf_inc_pp/(X_expl.shape[0])*100
avg_conf_inc = conf_inc/(X_expl.shape[0])*100

avg_win_pp = win_pp/(X_expl.shape[0])*100
avg_win = win/(X_expl.shape[0])*100

In [ ]:
print('average Confidence drop % (lower is better)')
print('Grad-CAM++', round(avg_conf_drop_pp, 2))
print('Grad-CAM', round(avg_conf_drop,2))

In [ ]:
print('Confidence inc % (higher is better)')
print('Grad-CAM++',avg_conf_inc_pp)
print('Grad-CAM',avg_conf_inc)

In [ ]:
print('Win % (higher is better)')
print('Grad-CAM++',round(avg_win_pp,2))
print('Grad-CAM',round(avg_win,2))


In [ ]:
conf_drop_pp = 0
conf_drop = 0

conf_inc_pp = 0
conf_inc = 0

win_pp = 0
win = 0
X_expl = X_test
y_int_expl = y_int_test
print(X_expl.shape)
for i in range(X_expl.shape[0]):
    print(i)
    gradCAM_pp_activations = gradCAMplusplus(img_array = np.expand_dims(X_expl[i], axis=0), model = model, grad_conv_model = grad_model, classifier_layer_names= classifier_layer_names)
    gradCAM_activations = gradCAM(img_array = np.expand_dims(X_expl[i], axis=0), model = model, grad_conv_model = grad_model, classifier_layer_names= classifier_layer_names)
    
    print('************************************')
    print('Grad-CAM ++ Input for evaluation:')
    gradCAM_pp_expl = explain_gradCAM(activations = gradCAM_pp_activations, img_index = i)
    print('Grad-CAM Input for evaluation:')
    gradCAM_expl = explain_gradCAM(activations = gradCAM_activations, img_index = i)
    
    conf_drop_pp_i = calculate_conf_drop(model = model, img_index = i, explanation = gradCAM_pp_expl)
    conf_drop_pp += conf_drop_pp_i

    conf_drop_i = calculate_conf_drop(model = model, img_index = i, explanation = gradCAM_expl)
    conf_drop += conf_drop_i
    
    conf_inc_pp_i = calculate_conf_inc(model = model, img_index = i, explanation = gradCAM_pp_expl)
    conf_inc_pp += conf_inc_pp_i

    conf_inc_i = calculate_conf_inc(model = model, img_index = i, explanation = gradCAM_expl)
    conf_inc += conf_inc_i
    
    win_pp_i, win_i = calculate_conf_win(model = model, img_index = i, explanation_pp = gradCAM_pp_expl, explanation = gradCAM_expl)
    win_pp += win_pp_i
    win += win_i
    
avg_conf_drop_pp = conf_drop_pp*100/(X_expl.shape[0])
avg_conf_drop = conf_drop*100/(X_expl.shape[0])

avg_conf_inc_pp = conf_inc_pp/(X_expl.shape[0])*100
avg_conf_inc = conf_inc/(X_expl.shape[0])*100

avg_win_pp = win_pp/(X_expl.shape[0])*100
avg_win = win/(X_expl.shape[0])*100

In [ ]:
print('average Confidence drop % (lower is better)')
print('Grad-CAM++', round(avg_conf_drop_pp, 2))
print('Grad-CAM', round(avg_conf_drop,2))

In [ ]:
print('Confidence inc % (higher is better)')
print('Grad-CAM++',round(avg_conf_inc_pp,2))
print('Grad-CAM',round(avg_conf_inc,2))

In [ ]:
print('Win % (higher is better)')
print('Grad-CAM++',round(avg_win_pp,2))
print('Grad-CAM',round(avg_win,2))


Compare Heatmaps with Silhouttenkoeffizient

In [ ]:
#get human heatmaps for image
def extract_heatmaps_for_image(image_id, user_folders_path):
    image_heatmaps = {}
    user_folders = os.listdir(user_folders_path)
    for user_folder in user_folders:
        heatmap_path = os.path.join(user_folders_path, user_folder, f"{image_id}.npy")
        if os.path.exists(heatmap_path):
            heatmap = np.load(heatmap_path)
            image_heatmaps[user_folder] = heatmap
            
    return image_heatmaps

#calculate wasserstein distance for one pair of heatmaps
def calculate_distance(heatmap1, heatmap2):
    heatmap1 = heatmap1.flatten()
    heatmap2 = heatmap2.flatten()
    similarity = wasserstein_distance(heatmap1, heatmap2)
    return similarity

#calculate a_j (average wasserstein distance of human heatmap to all other human heatmaps)
def average_distance_to_others(heatmap, all_heatmaps):
    distances = []
    for other_heatmap in all_heatmaps:
        if np.array_equal(heatmap, other_heatmap):
            continue 
        distance = calculate_distance(heatmap, other_heatmap)
        distances.append(distance)
    average_distance = np.mean(distances)
    return average_distance

#calculate b_j (wasserstein distance of human heatmap to XAI heatmap)
def distance_to_XAI(heatmap, xai_heatmaps):
    distances = []
    for other_heatmap in xai_heatmaps:
        distance = calculate_distance(heatmap, other_heatmap)
        distances.append(distance)
    return distances

def distance_to_others(heatmap, all_heatmaps):
    distances = []
    for other_heatmap in all_heatmaps:
        if np.array_equal(heatmap, other_heatmap):
            continue 
        distance = calculate_distance(heatmap, other_heatmap)
        distances.append(distance)
    return distances

In [ ]:
user_folders_path = 'XXX'
filepath_GradCAM = 'XXX'
all_files = os.listdir(filepath_GradCAM)
filtered_files = [file for file in all_files if file.endswith("pp.npy")]
filtered_files = [file.split('_GradCAMpp')[0] for file in filtered_files]

distribution_of_silhoutten_pp = []
distribution_of_silhoutten = []
i = 0
for stimuli in filtered_files:
    i += 1
    print(i, ": Calculating for image:", stimuli)
    image_id = stimuli
    #get GradCAM and GradCAM++ explanations
    gradCAM_pp_attention = np.load(filepath_GradCAM+'/'+image_id+'_GradCAMpp.npy')
    gradCAM_attention = np.load(filepath_GradCAM+'/'+image_id+'_GradCAM.npy')
    gradCAM_pp_up = upsample_gradCAM(gradCAM_pp_attention)
    gradCAM_up = upsample_gradCAM(gradCAM_attention)
    xai_heatmaps = {}
    xai_heatmaps["gradCAM_pp"] = gradCAM_pp_up
    xai_heatmaps["gradCAM"] = gradCAM_up

    image_heatmaps = extract_heatmaps_for_image(image_id, user_folders_path)
    
    silhoutten_koeffizient_GradCAM = []
    silhoutten_koeffizient_GradCAM_pp = []
    
    for user_id, heatmap in image_heatmaps.items():
        average_distance_human = average_distance_to_others(heatmap, list(image_heatmaps.values()))
        #print(f"Average similarity of heatmap for {user_id} to all others: {average_distance_human}")
        distance_human_to_XAI = distance_to_XAI(heatmap, list(xai_heatmaps.values()))
        #print(f"Similarity of heatmap for {user_id} to xai: {distance_human_to_XAI}")
        silhoutten_koeffizient_GradCAM_pp.append((distance_human_to_XAI[0]-average_distance_human)/max(distance_human_to_XAI[0],average_distance_human)) 
        silhoutten_koeffizient_GradCAM.append((distance_human_to_XAI[1]-average_distance_human)/max(distance_human_to_XAI[1],average_distance_human))
    
    distribution_of_silhoutten_pp.append(np.mean(silhoutten_koeffizient_GradCAM_pp))
    distribution_of_silhoutten.append(np.mean(silhoutten_koeffizient_GradCAM))
    
    


In [ ]:
user_folders_path = 'XXX'
filepath_GradCAM = 'XXX'
all_files = os.listdir(filepath_GradCAM)
filtered_files = [file for file in all_files if file.endswith("pp.npy")]
filtered_files = [file.split('_GradCAMpp')[0] for file in filtered_files]
i = 0
distribution_of_distances = []

for stimuli in filtered_files:
    i += 1
    print(i, ": Calculating for image:", stimuli)
    image_id = stimuli
    image_heatmaps = extract_heatmaps_for_image(image_id, user_folders_path)
    
    for user_id, heatmap in image_heatmaps.items():
        distribution_of_distances.append(distance_to_others(heatmap, list(image_heatmaps.values()))) 

In [ ]:
data = pd.read_excel('XAITrackingX11.xlsx')

In [ ]:
mean_human_XAIpp = np.mean(data['wasserstein distance ++'])
std_human_XAIpp = np.std(data['wasserstein distance ++'])

mean_human_XAI = np.mean(data['wasserstein distance'])
std_human_XAI = np.std(data['wasserstein distance'])

dist_of_inter_human_distances = np.array(list(itertools.chain.from_iterable(distribution_of_distances)))
mean_inter_human = np.mean(dist_of_inter_human_distances)
std_inter_human = np.std(dist_of_inter_human_distances)

plt.figure(figsize=(12, 6))

sns.set_theme(style="white")
sns.histplot(data['wasserstein distance ++'], label='AI-human (Grad-CAM++)', color='blue', kde=True)
sns.histplot(data['wasserstein distance'], label='AI-human (Grad-CAM)', color='orange', kde=True)
#sns.histplot(np.array([item for sublist in distribution_of_distances for item in sublist]), label='inter-human', color='red', kde=True)
sns.histplot(dist_of_inter_human_distances, label='inter-human', color='red', kde=True)
plt.fill_between([mean_human_XAIpp - std_human_XAIpp, mean_human_XAIpp + std_human_XAIpp], 0, 1300, color='blue', alpha=0.1, label='±1 STDEV (AI-human for Grad-CAM++)')
plt.fill_between([mean_human_XAI - std_human_XAI, mean_human_XAI + std_human_XAI], 0, 1300, color='orange', alpha=0.1, label='±1 STDEV (AI-human for Grad-CAM)')
plt.fill_between([mean_inter_human - std_inter_human, mean_inter_human + std_inter_human], 0, 1300, color='red', alpha=0.1, label='±1 STDEV (inter-human)')

plt.axvline(mean_human_XAIpp, color='blue', linestyle='--', label='average (AI-human for Grad-CAM++)')
plt.axvline(mean_human_XAI, color='orange', linestyle='--', label='average (AI-human for Grad-CAM)')
plt.axvline(mean_inter_human, color='red', linestyle='--', label='average (inter-human)')

plt.xlabel("Earth mover's distance")
plt.ylabel('Frequency')
plt.xlim(0,np.max(data['wasserstein distance ++']))
plt.ylim(0,1300)
plt.legend()
plt.title("Distribution of inter-human and AI-human Earth mover's distance")

plt.savefig('wasserstein_distance_interhuman_vs_humanXAI_vs_humanXAI.png', dpi=300)

plt.show()

In [ ]:
np.mean(data['wasserstein distance'])

In [ ]:
from scipy import stats

res_XAI_pp = stats.shapiro(data['wasserstein distance ++'])
res_XAI = stats.shapiro(data['wasserstein distance'])
res_human = stats.shapiro(dist_of_inter_human_distances)
print(res_XAI_pp)
print(res_XAI)
print(res_human)

In [ ]:
res = stats.levene(data['wasserstein distance ++'], dist_of_inter_human_distances)
res

In [ ]:
statistic, p_value = mannwhitneyu(data['wasserstein distance ++'], dist_of_inter_human_distances, alternative = 'greater')

print('********************')
print('Mann Whitney U test:')
print('********************')
print('is a nonparametric test of the null hypothesis that, for randomly selected values X and Y from two populations, the probability of X being greater than Y is equal to the probability of Y being greater than X. The Mann-Whitney U test is a nonparametric test of the null hypothesis that the distribution underlying sample x is the same as the distribution underlying sample y. It is often used as a test of difference in location between distributions.')
print()
print('p-value:', p_value)
print(statistic)

In [ ]:
statistic, p_value = mannwhitneyu(data['wasserstein distance'], dist_of_inter_human_distances, alternative = 'greater')

print('********************')
print('Mann Whitney U test:')
print('********************')
print('is a nonparametric test of the null hypothesis that, for randomly selected values X and Y from two populations, the probability of X being greater than Y is equal to the probability of Y being greater than X. The Mann-Whitney U test is a nonparametric test of the null hypothesis that the distribution underlying sample x is the same as the distribution underlying sample y. It is often used as a test of difference in location between distributions.')
print()
print('p-value:', p_value)
print(statistic)

In [ ]:
statistic, p_value = ttest_ind(data['wasserstein distance ++'], dist_of_inter_human_distances, alternative = 'greater')

print('********************')
print('two sample t test:')
print('********************')
print('This is a test for the null hypothesis that 2 independent samples have identical average (expected) values. This test assumes that the populations have identical variances by default.')
print()
print('p-value:', p_value)
print(statistic)

In [ ]:
sns.set_theme(style="white")
mean_GradCAM_pp = np.mean(distribution_of_silhoutten_pp)
std_GradCAM_pp = np.std(distribution_of_silhoutten_pp)

mean_GradCAM = np.mean(distribution_of_silhoutten)
std_GradCAM = np.std(distribution_of_silhoutten)
plt.figure(figsize=(12, 6))
sns.histplot(distribution_of_silhoutten, label='Grad-CAM', color='orange', kde=True, binwidth=0.04)
sns.histplot(distribution_of_silhoutten_pp, label='Grad-CAM++', color='blue', kde=True, binwidth=0.04)

plt.fill_between([mean_GradCAM_pp - std_GradCAM_pp, mean_GradCAM_pp + std_GradCAM_pp], 0, 50, color='blue', alpha=0.1, label='±1 STDEV (Grad-CAM++)')
plt.fill_between([mean_GradCAM - std_GradCAM, mean_GradCAM + std_GradCAM], 0, 50, color='orange', alpha=0.1, label='±1 STDEV (Grad-CAM)')
plt.xlabel('average silhouette metric')
plt.axvline(mean_GradCAM, color='orange', linestyle='dashed', linewidth=1, label='average (Grad-CAM)')
plt.axvline(mean_GradCAM_pp, color='blue', linestyle='dashed', linewidth=1, label='average (Grad-CAM++)')
plt.axvline(0, color='black', linestyle='dashed', linewidth=1, label='no clustering structure')
#plt.ylabel('Frequency')
plt.xlim(-0.1,1)
plt.ylim(0,45)
#plt.ylim(0,150)
plt.legend(loc = "upper left")
plt.title('Distribution of silhouette metric')
plt.plot()
plt.savefig('silhouette_metric.png', dpi=300)
plt.show()

In [ ]:
res_pp = stats.shapiro(distribution_of_silhoutten_pp)
res = stats.shapiro(distribution_of_silhoutten)
print(res_pp)
print(res)

In [ ]:
statistic, p_value = mannwhitneyu(distribution_of_silhoutten_pp, 0, alternative = 'greater')

print('********************')
print('Mann Whitney U test:')
print('********************')
print('is a nonparametric test of the null hypothesis that, for randomly selected values X and Y from two populations, the probability of X being greater than Y is equal to the probability of Y being greater than X. The Mann-Whitney U test is a nonparametric test of the null hypothesis that the distribution underlying sample x is the same as the distribution underlying sample y. It is often used as a test of difference in location between distributions.')
print()
print('p-value:', p_value)
print(statistic)

In [ ]:
statistic, p_value = mannwhitneyu(distribution_of_silhoutten, 0, alternative = 'greater')

print('********************')
print('Mann Whitney U test:')
print('********************')
print('is a nonparametric test of the null hypothesis that, for randomly selected values X and Y from two populations, the probability of X being greater than Y is equal to the probability of Y being greater than X. The Mann-Whitney U test is a nonparametric test of the null hypothesis that the distribution underlying sample x is the same as the distribution underlying sample y. It is often used as a test of difference in location between distributions.')
print()
print('p-value:', p_value)
print(statistic)

In [ ]:
np.mean(distribution_of_silhoutten)

In [ ]:
np.std(distribution_of_silhoutten)

In [ ]:
np.median(distribution_of_silhoutten)